## Simple Chain (LCEL - LangChain Expression Language)
  The modern way using | operator:

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")
else:
   raise ValueError("OPENAI_API_KEY NOT FOUND") 

chain = (
    ChatPromptTemplate.from_template("Explain {topic} in simple terms")
    | ChatOpenAI(model="gpt-5-mini")
    | StrOutputParser()
)

result = chain.invoke({"topic": "blockchain"})
print(result)

API KEY VARAIBLE EXIST
Short answer
A blockchain is a shared, tamper-resistant digital ledger that records transactions in a way that many computers agree on. Once something is added, it’s very hard to change, and everyone with access can see the same history.

Simple analogy
- Think of a blockchain like a public notebook that everyone can copy. Whenever people agree a page is correct, it’s glued into a binder. Everyone replaces their old notebook with the new glued binder, so all copies stay the same.

How it works (step-by-step, simply)
1. Someone creates a transaction (e.g., Alice sends 1 coin to Bob).
2. That transaction is broadcast to a network of computers (nodes).
3. Nodes check the transaction is valid (Alice has the coin and the signature is correct).
4. Valid transactions are grouped into a “block” (like a page in the notebook).
5. The network uses a process (consensus) to agree which block is added next. Different systems do this differently (examples: proof-of-work or proo

## Sequential Chain (Multiple Steps)
## Chain the output of one LLM call into the next:

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

# Step 1: Generate patient symptoms summary
symptoms_chain = (
    ChatPromptTemplate.from_template(
        "Summarize these symptoms in medical terms: {symptoms}"
    )
    | llm
    | StrOutputParser()
)

# Step 2: Generate treatment recommendation
treatment_chain = (
    ChatPromptTemplate.from_template(
        "Given this medical summary: {summary}, recommend treatment"
    )
    | llm
    | StrOutputParser()
)

# combine chain 
  # ← Step 1: Creates RunnableMap
 # ← Step 2: Pipes its OUTPUT to treatment_chain
full_chain = { "summary": symptoms_chain } | treatment_chain

result = full_chain.invoke({"symptoms":"fever, cough, fatigue for 3 days"})
print(result)

Suggested chart/HPI phrasing (pick/modify as appropriate)
- Quick note for triage: "3-day history of fever, cough, and generalized fatigue."
- HPI for chart: "Acute onset (3 days) of fever, cough, and malaise. Patient reports measured fever of 38.3°C at home and a dry cough; denies dyspnea, pleuritic chest pain, or sputum production."
- Alternative if subjective/productive: "Acute onset (3 days) of subjective fever, productive cough with yellowish sputum, and fatigue."

Key differential (most to least likely)
- Viral upper respiratory infection (including COVID-19, influenza)
- Acute bronchitis
- Community-acquired pneumonia (less likely if no dyspnea, focal exam findings, or hypoxia)
- Less likely: pertussis, atypical pathogens, noninfectious causes

Recommended initial management (outpatient, clinically stable)
1. Symptomatic care
   - Antipyretic/analgesic: acetaminophen or ibuprofen per product dosing for fever/aches.
   - Hydration, rest, throat lozenges/saline nasal irrigation.
 

##  Router Chain (Conditional Logic)
## Route to different chains based on input:

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini")

# Different chains for different departments
pediatric_chain = (
    ChatPromptTemplate.from_template("Pediatric advice for: {query}")
    | llm
    | StrOutputParser()
)

adult_chain = (
    ChatPromptTemplate.from_template("Adult care advice for: {query}")
    | llm
    | StrOutputParser()
)

# Router logic
def route_by_age(input_data):
    if input_data["age"] < 18:
        return pediatric_chain
    return adult_chain

# Use it
chain = route_by_age({"age": 25, "query": "HIV treatment"})
result = chain.invoke({"query": "HIV treatment options"})
print(result)

Brief overview
- Goal of HIV treatment: suppress plasma HIV RNA to “undetectable” and keep it there (prevents disease progression and stops sexual transmission).
- Recommendation: start antiretroviral therapy (ART) for all people with HIV as soon as possible after diagnosis (same‑day start often appropriate) unless there is a clear reason to delay.
- Choice of regimen is individualized by prior ART exposure, baseline viral load, CD4 count, resistance test results, coinfections (HBV, TB), pregnancy or pregnancy potential, drug interactions, kidney/hepatic function, and patient preference.

Baseline tests before or at ART start
- HIV RNA (viral load) and CD4 count
- HIV genotype (resistance) test if possible (guides regimen choice)
- Hepatitis B surface antigen, hepatitis B surface antibody, hepatitis C antibody/RNA
- HLA‑B*5701 (if abacavir is considered)
- Pregnancy test for people who can be pregnant
- Renal function (creatinine, eGFR), liver tests, fasting lipids and glucose
- STI sc

## Router Chain (Conditional Logic)
### Route to different chains based on input

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

pediatric_Chain = (
    ChatPromptTemplate.from_template("Pediatric advise for: {query} ")
    | llm | StrOutputParser())


adult_Chain = (ChatPromptTemplate.from_template("Adult care advise for: {query} ") 
 | llm | StrOutputParser())

def route_by_age(input_data): 
    if input_data["age"] > 18:
       return adult_Chain
    return pediatric_Chain

chain = route_by_age({"age": 25, "query": "HIV treatment"})
result = chain.invoke({"query": "HIV treatment options"})
print(result)




## Example: Patient Assessment Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from pydantic import Field
from pydantic import BaseModel
from langchain_openai import ChatOpenAI


class TreatmentRecommendation(BaseModel):
    recommendation: str = Field(description="Treatment recommendation")
    urgency: str = Field(description="low, medium, or high")
    follow_up_days: int = Field(description="Days until follow-up")
    notes: str = Field(description="Additional clinical notes")



analysis_chain = (
    ChatPromptTemplate.from_template("""
    Analyze this HIV patient data:
    
    Patient ID: {patient_id}
    CD4 Count: {cd4_count}
    Viral Load: {viral_load}
    Current Regimen: {regimen}
    Adherence: {adherence}
    
    Provide a clinical assessment.
    """)
    | llm
    | StrOutputParser()
)

# Step 2: Generate structured recommendation
recommendation_chain = (
    ChatPromptTemplate.from_template("""
    Based on this assessment: {assessment}
    
    Provide treatment recommendation in JSON format:
    {{
        "recommendation": "specific action to take",
        "urgency": "low/medium/high",
        "follow_up_days": number,
        "notes": "additional notes"
    }}
    """)
    | llm
    | JsonOutputParser(pydantic_object=TreatmentRecommendation)
)


full_assessment_chain = {
     {
        "assessment": analysis_chain,
        "patient_id": lambda x: x["patient_id"]
    }
} | recommendation_chain

patient_data = {
    "patient_id": "LAM12345",
    "cd4_count": 180,
    "viral_load": 75000,
    "regimen": "TDF/3TC/EFV",
    "adherence": "Good"
}

result = full_assessment_chain.invoke(patient_data)
print(result)

ModuleNotFoundError: No module named 'tables'